## Building the dataset that will be input into the model

In [1]:
import findspark
import pandas as pd
findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkConf

# for shared metastore (shared across all users)
spark = SparkSession.builder.appName("Building dataset").config("hive.metastore.uris", "thrift://bialobog:9083", conf=SparkConf()).getOrCreate() \

# for local metastore (your private, invidivual database) add the following config to spark session
spark.sql("USE 2023_04_01")

DataFrame[]

In [2]:
import pyspark.pandas as ps
from pyspark.sql.functions import lit,col
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
#from boruta import BorutaPy
#from fredapi import Fred
from sklearn.linear_model import Lasso
from sklearn.model_selection import TimeSeriesSplit
import csv
from pyspark.sql import functions as F
from functools import reduce
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
from pyspark.ml.regression import LinearRegression
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, CrossValidatorModel
from CreateDataset import get_features_all_stocks_seq, get_full_series_stocks

/opt/spark/python/pyspark/pandas/__init__.py:50: UserWarning: 'PYARROW_IGNORE_TIMEZONE' environment variable was not set. It is required to set this environment variable to '1' in both driver and executor sides if you use pyarrow>=2.0.0. pandas-on-Spark will set it for you but it does not work if there is a Spark context already launched.
  warnings.warn(


In [3]:
df = pd.read_csv('imploded_stocks_price.csv', index_col=False)
full_series_stocks = get_full_series_stocks(df) #this gets the stocks that have data since 2001
filtered_df = df[df['fsym_id'].isin(full_series_stocks)]
filtered_df = get_features_all_stocks_seq(filtered_df)

[Row(fsym_id='B01DPB-R', date=datetime.date(2001, 12, 31), ff_assets_gr=16.344051186523, ff_earn_yld=3.17269, ff_net_inc_basic_aft_xord=484.0, ff_net_inc_dil_bef_unusual=505.0, ff_net_inc_basic_beft_xord=484.0, ff_wkcap=759.2, ff_oper_inc_aft_unusual=744.6, ff_net_inc_bef_xord_gr=29.3080416777986, ff_fcf_yld=3.49776, ff_non_oper_exp=-60.8, ff_net_inc_dil=484.0, ff_net_inc_dil_aft_xord=484.0, ff_oper_inc_gr=-3.00525920360631, ff_ut_non_oper_inc_oth=-30.8), Row(fsym_id='B01DPB-R', date=datetime.date(2002, 12, 31), ff_assets_gr=27.46954403063, ff_earn_yld=6.48298, ff_net_inc_basic_aft_xord=611.8, ff_net_inc_dil_bef_unusual=668.92, ff_net_inc_basic_beft_xord=611.8, ff_wkcap=1437.4, ff_oper_inc_aft_unusual=917.1, ff_net_inc_bef_xord_gr=26.4049586776859, ff_fcf_yld=9.56695, ff_non_oper_exp=-124.8, ff_net_inc_dil=611.8, ff_net_inc_dil_aft_xord=611.8, ff_oper_inc_gr=28.9310611928737, ff_ut_non_oper_inc_oth=-43.2), Row(fsym_id='B01DPB-R', date=datetime.date(2003, 12, 31), ff_assets_gr=26.677953

NameError: name 'length_freq_info' is not defined

In [ ]:



def get_macro_features():
    # fred_key = 'bdfdde3b7a21b7d528011d17996b0b8e'
    # fred = Fred(api_key=fred_key)
    # cpi = fred.get_series(series_id='CPIAUCSL')
    # cpi_change = cpi.pct_change()
    # unemp = fred.get_series(series_id='UNRATE')
    # gdp = fred.get_series(series_id='GDP')
    # gdp_change = gdp.pct_change()
    # df = pd.DataFrame({'CPI_change': cpi_change,'Unemployment_Rate': unemp,'GDP_change': gdp_change})
    # df.to_csv('macro.csv')
    df = pd.read_csv('macro.csv')
    return df


def train_test(df):
    print("Number of records: ", df.count())
    df.show()
    df = df.filter(reduce(lambda acc, column: acc & (F.size(col(column)) == 23), df.columns[1:-1], lit(True)))
    # print(filtered_df.count())

    # average_lengths = df.agg(*[(F.avg(F.size(col(column))).alias(f'avg_length_{column}')) for column in df.columns[1:-1]])
    
    # test = padded_df.select('ff_non_oper_exp').filter(col('fsym_id')=='RTTY5P-R').collect()[0]
    # print(test['ff_non_oper_exp'], len(test['ff_non_oper_exp']))

    #need to decide whether to only include stocks that started from 2000, or include just from e.g. 2019
    #temporary measure - replace with 0
    #try imputer?
    #look into masking
    features = df.columns[1:-1]
    list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
    for f in features:
        df = df.withColumn(f, list_to_vector_udf(f))
    df.show(2)
    vector_assembler = VectorAssembler(inputCols=features, outputCol="features_vector")
    df_assembled = vector_assembler.transform(df)
    
    lr = LogisticRegression(featuresCol="features_vector", labelCol="label")

    # pipeline = Pipeline(stages=[vector_assembler, lr])
    
    paramGrid = ParamGridBuilder() \
        .addGrid(lr.regParam, [0.01, 0.1]) \
        .addGrid(lr.elasticNetParam, [0.0, 0.5]) \
        .addGrid(lr.maxIter, [10, 20]) \
        .build()

    evaluator = BinaryClassificationEvaluator(labelCol="label", rawPredictionCol="rawPrediction", metricName="areaUnderROC")

    crossval = CrossValidator(estimator=lr,
                              estimatorParamMaps=paramGrid,
                              evaluator=evaluator,
                              numFolds=5) 

    cvModel = crossval.fit(df_assembled)

    avg_metrics = cvModel.avgMetrics

    for i, acc in enumerate(avg_metrics):
        print(f"Fold {i + 1} - Validation Accuracy: {acc}")

    best_model = cvModel.bestModel

    predictions = best_model.transform(df_assembled)
    predictions.select('fsym_id', 'label', 'prediction').show(100)
    tp = predictions.filter((predictions.label == 1) & (predictions.prediction == 1)).count()
    tn = predictions.filter((predictions.label == 0) & (predictions.prediction == 0)).count()
    fp = predictions.filter((predictions.label == 0) & (predictions.prediction == 1)).count()
    fn = predictions.filter((predictions.label == 1) & (predictions.prediction == 0)).count()
    
    print(f"True Positives: {tp}")
    print(f"True Negatives: {tn}")
    print(f"False Positives: {fp}")
    print(f"False Negatives: {fn}")

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1_score = 2 * (precision * recall) / (precision + recall)

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1 Score: {f1_score}")

#     # Filter and display predictions for label 1
#     predictions.filter(predictions['label'] == 1).select("fsym_id", "label", "prediction").show()
# from sklearn.model_selection import train_test_split, GridSearchCV
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import roc_auc_score

def test_pandas(df1):
    df1 = df1.toPandas()
    exclude_columns = ['fsym_id', 'label']
    df = df1[df1.loc[:, ~df1.columns.isin(exclude_columns)].apply(lambda row: all(len(cell) == 23 for cell in row), axis=1)]
    print("Number of records: ", len(df))
    X = df.drop(exclude_columns, axis=1)
    y = df['label']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    lr = LogisticRegression()

    param_grid = {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1.0, 10.0],
        'max_iter': [100, 200, 300]
    }

    grid_search = GridSearchCV(lr, param_grid, cv=5, scoring='roc_auc')

    grid_search.fit(X_train, y_train)
    
    best_model = grid_search.best_estimator_

    predictions = best_model.predict(X_test)

    auc = roc_auc_score(y_test, predictions)

    print(f"Area under the ROC curve (AUC): {auc}")
    print("Best model hyperparameters:")
    print(grid_search.best_params_)
    
    #extra_df= df[df.apply(lambda row: any(len(cell) != 23 for cell in row), axis=1)]
    
    
    
    
#create_dataset('FF_ADVANCED_DER_AF')

train_test(filtered_df)
#test_pandas(df)

In [ ]:
#spark.sql("SELECT MIN(a.p_date) FROM fp_basic_prices a LEFT JOIN sym_ticker_region s ON s.fsym_id = a.fsym_id WHERE s.ticker_region = 'AACQU-US' ").show()